In [1]:
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

In [2]:
data_file = pd.read_csv("../TAMU_FINAL_DATASET_2018.csv")

In [13]:
# only select the numeric data
df = data_file.select_dtypes(include=['float64','int64'])
categorical_data = data_file.select_dtypes(exclude=['float64','int64'])
categorical_df = pd.DataFrame()
for column in categorical_data.columns:
    dummies = pd.get_dummies(categorical_data[column], prefix=column)
    categorical_df = pd.concat([categorical_df, dummies], axis=1)
#df = pd.concat([df['AMI_FLAG'], categorical_df], axis=1)
df = pd.concat([df, categorical_df], axis=1)

# drop the null value 

print(df.shape)
df = df.dropna()
print(df.shape)

(100000, 469)
(74762, 469)


In [14]:
sum(df['AMI_FLAG'] == 1)
df_AMI = df[df['AMI_FLAG'] == 1]
df_NOAMI = df[df['AMI_FLAG'] == 0]
df_NOAMI_sample = df_NOAMI.sample(2726)
print(df_AMI.shape)
print(df_NOAMI_sample.shape)

(1992, 469)
(2726, 469)


In [24]:
df_small = pd.concat([df_AMI, df_NOAMI_sample], axis = 0)


# define the target we would like to predict 
y = df_small['AMI_FLAG']

X = df_small.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)

# split training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=42)


# define the target we would like to predict 
y = df['AMI_FLAG']

X = df.loc[:, df.columns != 'AMI_FLAG']

# sanity check on X, y dimensions 
print(X.shape)
print(y.shape)


X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, test_size = 0.33, random_state=42)

(4718, 468)
(4718,)
(74762, 468)
(74762,)


In [26]:
# construct linear model using sklearn 
lm = LogisticRegression(penalty='l1')
model = lm.fit(X_train, y_train)

pred_train = lm.predict(X_train_all)
pred_test = lm.predict(X_test_all)

#print(pred_train[:10])
#print(y_train[:10])

#print(pred_test[:10])
#print(y_test[:10])

/Users/Jimin/anaconda3/envs/NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
lm = LogisticRegression(penalty='l1')
model = lm.fit(X_train_all, y_train_all)

pred_train_all = lm.predict(X_train_all)
pred_test_all = lm.predict(X_test_all)

/Users/Jimin/anaconda3/envs/NLP/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
# logistic regression result 

#print(np.count_nonzero(pred_train))
#print(np.count_nonzero(y_train))

#print(np.count_nonzero(pred_test))
#print(np.count_nonzero(y_test))

print(accuracy_score(y_train_all, pred_train))
print(accuracy_score(y_test_all, pred_test))

print(accuracy_score(y_train_all, pred_train_all))
print(accuracy_score(y_test_all, pred_test_all))


0.783629466959473
0.782871271076524
0.9740666799760431
0.9713035019455253


In [ ]:
def eliminate_covariate(df):
    for i in range(len(df.columns)):
        for j in range(len(df.columns)):
            if i != j:
                if np.corrcoef(df.iloc[:,j], df.iloc[:,i])[1,0] > 0.8:
                    print(df.columns[i], df.columns[j])